# Test The Data Loader


In [1]:
# Debugs and Config
# Imports
from grakel.datasets import fetch_dataset
from sklearn.model_selection import train_test_split
import joblib
# Imports
import networkx as nx
import numpy as np
from grakel.graph import Graph # To convert NetworkX to GraKeL Graph objects
import traceback
import os
LOCAL_DATA_PATH = 'Datasets' # Make sure this points to your extracted MUTAG folder
DOWNLOAD_SOURCE = 'TUD'
DATASET_NAME = 'NONE' # Placeholder, will be set later
DEBUG = True # Set to False to disable debug prints

In [2]:
# import the Dataset class
from Dataset import Dataset


In [3]:
MUTAG = Dataset(name='MUTAG', source='TUD', domain='molecular')

Loading MUTAG into NetworkX from Datasets\TUD\MUTAG...


c:\Python311\Lib\site-packages\grakel\graph.py:314: UserWarning: changing format from "adjacency" to "all"
  warnings.warn('changing format from "adjacency" to "all"')


In [ ]:
display(MUTAG.attributes())


{'name': 'MUTAG',
 'source': 'TUD',
 'domain': 'molecular',
 'num_graphs': 188,
 'num_classes': 2,
 'has_node_labels': True,
 'has_edge_labels': False,
 'mean_nodes': 17.930851063829788,
 'mean_edges': 19.79255319148936}

In [5]:
# split the dataset into training and testing sets
X_train, X_test, y_train, y_test = MUTAG.train_test_split(test_size=0.2, random_state=42)

In [6]:
print("Training set size:", len(X_train))
print("Testing set size:", len(X_test))

Training set size: 150
Testing set size: 38


In [7]:
# display the attributes of the dataset
if DEBUG:
    print("Dataset Attributes:")
    print(MUTAG.attributes())

Dataset Attributes:
{'name': 'MUTAG', 'source': 'TUD', 'domain': 'molecular', 'num_graphs': 188, 'num_classes': 2, 'has_node_labels': True, 'has_edge_labels': False, 'mean_nodes': 17.930851063829788, 'mean_edges': 19.79255319148936}


# tying to train model on it.


In [10]:
from grakel.kernels import WeisfeilerLehman
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


In [12]:
display(y_train)
display(y_test)

ValueError: purpose is either "adjacency" of "dictionary"

ValueError: purpose is either "adjacency" of "dictionary"

In [ ]:
 # --- Initialize WL-Kernel and Compute Kernel Matrices ---
print("\nInitializing WeisfeilerLehman Kernel and computing kernel matrices...")
wl_kernel = WeisfeilerLehman(n_iter=5, normalize=True) # You can tune n_iter and normalize

# K_train will be a (n_train_samples, n_train_samples) matrix
K_train = wl_kernel.fit_transform(X_train)
print(f"Shape of K_train: {K_train.shape}")

# K_test will be a (n_test_samples, n_train_samples) matrix
K_test = wl_kernel.transform(X_test)
print(f"Shape of K_test: {K_test.shape}")

# --- Train and Test SVM Classifier ---
print("\nTraining SVM Classifier with precomputed kernel...")
svm_classifier = SVC(kernel="precomputed", C=1.0, random_state=42) # C is the regularization parameter
svm_classifier.fit(K_train, y_train)

print("Making predictions on the test set...")
y_pred = svm_classifier.predict(K_test)

# --- Evaluate Performance ---
accuracy = accuracy_score(y_test, y_pred)
print(f"\nAccuracy of SVM with WL-Kernel: {accuracy:.4f}")


Initializing WeisfeilerLehman Kernel and computing kernel matrices...
Shape of K_train: (150, 150)
Shape of K_test: (38, 150)

Training SVM Classifier with precomputed kernel...


ValueError: Unknown label type: unknown. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.